In [1]:
def Mod(a, b):
    return a%b

def mod_binary(k, g, p):
    y=1
    k = str(bin(k))[2:]
    for i in k:
        if i=='1':
            y = Mod(Mod(y**2, p)*g, p)
            # print(y)
        else:
            y = Mod(y**2, p)
            # print(y)
    return y

def ex_euclid(a, b):
    A = [a, b]
    X = [1, 0]
    Y = [0, 1]
    i=1
    while A[-1]!=0:
        q = A[i-1]//A[i]
        a_next = A[i-1]-A[i]*q
        x_next = X[i-1]-q*X[i]
        y_next = Y[i-1]-q*Y[i]
        A.append(a_next)
        X.append(x_next)
        Y.append(y_next)
        i+=1
    
    ans = []
    ans.append((X[i-1], Y[i-1]))
    d = A[i-1]
    x = X[i-1]
    y = Y[i-1]
    return x, y, d

def inv(a, n):
    x, y, d = ex_euclid(a, n)
    inv = x%n
    # print("inv =", inv)
    # print("inv*a mod n =", (inv*a)%n)
    return inv

In [33]:
import hashlib, math
# ハッシュ関数Shake128によるハッシュ値の計算
# Input: メッセージ m (通常の文字列), ベースポイントの位数 l (整数)
# Output: shake128による m のハッシュ値 (16進文字列)
def shake128(m, l):
    return hashlib.shake_128(m.encode('utf-8')).hexdigest(math.ceil(l.bit_length()/8)-1)

def dsa_sign_gen_key(p, g, l, sk):
    #Input: 法 mod：p, ベースポイント base point：g, ベースポイントの位数 base point number position：ℓ, 秘密鍵secret key：sk
    #Output: 公開鍵 public key: pk = gsk mod p
    return mod_binary(sk, g, p)

def dsa_sign_gen(m, g, l, r, p, sk):
    u = mod_binary(r, g, p) % l
    m_prime = shake128(str(m), l)
    v = (inv(r, l)*(int(m_prime, 16) + sk*u)) % l
    return (u, v)

def dsa_sign_verify(m, sigma, l, g, p, pk):
    print("m =", m)
    print("start verification")
    u, v = sigma
    m_prime = shake128(str(m), l)
    v_inv = inv(v, l)
    u_prime = (mod_binary(int(m_prime, 16)*v_inv, g, p) * mod_binary(u*v_inv, pk, p)) % p % l
    print("m' =", m_prime)
    if u_prime == u:
        return 'OK'
    else:
        return 'NG'

def elgamal_enc(m, g, p, r, pk):
    u = mod_binary(r, g, p)
    c = Mod(mod_binary(r, pk, p)*int(m), p)
    return (u, c)

def elgamal_dec(u, c, p, sk):
    m = Mod(c*inv(mod_binary(sk, u, p), p), p)
    return m

In [27]:
def hybrid_key_gen(p, g, sk):
    #Input: 法 mod：p, ベースポイント base point：g, 秘密鍵 secret key：sk
    #Output: 公開鍵 public key: pk = gsk mod p
    return mod_binary(sk, g, p)

def hybrid_enc_sign_gen(m, g, l, r, p, sk, pk):
    # Input: メッセージ (通常の文字列) message (string)：m, 秘密鍵 secret key：sk, 公開鍵 public key：pk, ベースポイン
    # ト base point：g, ベースポイントの位数 base point number position：ℓ, 乱数 (整数) random number (integer)：
    # r, 法 mod：p
    # Output: m の暗号文 (整数の配列) m cyphertext(integer array)：[c1 = gr, c2 = pkrm], 署名 (整数の配列) signa-
    # ture(integer array)：[u, v]
    return elgamal_enc(m, g, p, r, pk), dsa_sign_gen(m, g, l, r, p, sk)

def hybrid_enc_sign_verify(C, sigma, sk, l, g, p, pk):
    # Input: m の暗号文 ciphertext of m：C = [c1, c2], m の署名 signature of m：sigma=[u, v], ベースポイント base point：
    # g, ベースポイントの位数 base point number position：ℓ, 公開鍵 public key：pk, 秘密鍵 secret key：sk, 法 mod：p
    # Output: C の復号結果 m, 署名が正しければ True, 誤っていれば False C decryption of m, If signatures are correct
    # True, if wrong False.
    m = elgamal_dec(C[0], C[1], p, sk)
    return m, dsa_sign_verify(m, sigma, l, g, p, pk)

In [41]:
import binascii
def str_to_int(m):
    # Input: 平文 (Unicode 文字列) Plaintext (Unicode string)：m
    # Output: message を変換した文字列 Converted integer.
    return int(binascii.hexlify(m.encode('unicode-escape')), 16)

def int_to_str(m):
    return binascii.unhexlify(hex(m)[2:]).decode('unicode-escape')

sk = 1033518767171415399648596791610869410243344457909
p4 = 141108755332974711606815218263958123381184588212061018448136404826965889433079453789166218230378522228564058127860367190611065605375025546257531489369344062782521806978218808940091447658298351853603237069980597505163602473095615670998464391197300372933147772009493823037167642459378452731092557170909406945309
g4 = 79207621787760038235763239269746451281552097558625763440050213854787240633084667257397421010854631623596917364929357681934505810865796708268321894884518347711092708958596829555918931536779252059763083320084867242870421484137196365244257388686997557137455046446997809953054632950185678637137955632299024284915
l4 = 1362115923099293242369922261305212343561846087883
r = 123456789

m = "standard course"

pk = hybrid_key_gen(p4, g4, sk)
print("pk:", pk)
pk_hex = hex(pk)
pk_kaiming=19608043294720164289108387353969576654624887844042068145593448178357480627933038987187231885796058369014166751878452899160356611903874053585553570392175730973735746091407698828972054492478952763667377416798291792362717792178902266696603863357395476340030920620273896270718742060121259471269322710546911544604
print("pk:", pk_hex)
C, sigma = hybrid_enc_sign_gen(str_to_int(m), g4, l4, r, p4, sk, pk_kaiming)
# C, sigma = hybrid_enc_sign_gen(m, g4, l4, r, p4, sk, pk_kaiming)
C_hex = (hex(C[0]), hex(C[1]))
sigma_hex = (hex(sigma[0]), hex(sigma[1]))
print("C:", C_hex)
print("sigma:", sigma_hex)

#####################
sk_kaiming = 1026017247403651479879862802368593908481232566634
m_dec, verify = hybrid_enc_sign_verify(C, sigma, sk_kaiming, l4, g4, p4, pk)
print("m_dec:", int_to_str(m_dec))
print("verify:", verify)

pk: 136150644009302630913702582649620119533778702615863555732256280557674421741884034673276135347141292002359672782520907093322852774854417957045361400789785858622001356628803010317554148247555192557079140987775463521577124974604015149316864906271422014141440740595727288567824404704464590931546439601559608715228
pk: 0xc1e28c44dafdb19be3cbd11950eeb66c2f4218c9b7177c049228788f58a010d8df4d194b84f83ecaee7f2720c505555c01323bebe89ce61f40ffd23019e40eb79e09086c31e33f1be4ad67729b975522eea6b0de0c9f13cb3b26cc297e6b8909bbb05a1b7bb6faa1c0776010a8e812e771eb0523bc704209bf0b8413445767dc
C: ('0x29820f3ab72f3993be57bfc36329dc4b631472f28bbfd45166a3656beef034a293781394624be3b2bb79a760b8e2c4eeacb2783b8c2ced3c68d6736c18dd6f6859c917fb947f8175b8656deb291c07c69a3fc3b05d3919b61f3b2650bb39326e272a6719665b278b78b24f34545f8aa098ce0d972e44729229f3c8e608426fb5', '0x16f52ffcb4d28cd787504ac653b5aacc740377d35f649df2704c41f4e1936a48d69d309f739e1eb052d24a9263050bea49e9e4f0331977093062ca1498557a920a6323d099b979628237401b24

In [43]:
# C_ta = (107006444221240209869002789834054283302949481099909127334146786004394095146588564045959885616224697566046579593370132777499146192862578771490260903308481943553195026398254907876712386980948791499702273297286948241536887860658299988693468139784997095660679401337044599203134732868657575692121953094743824317924, 105692554720373874918779975785057372822507533409217950659832106542109637204878945604002377667022023258227092571839033721292889238756663962090067601283593809617980467916731426553156000030672600539000684954958493386194293522039704206458391780959649743380782202570907611227216301145021092226103761137751755235440)
# sigma_ta = (310321584571945438057228348941480366965541514509, 483760452440746037711116508009250714501137031273)
C_ta = (105850165840080368917580071441014929069113949988334618277193149803280312657458410653029794796265344667190488507074548868463913256127941917800876735467643436215187409393813867541358835841606632753322097939560587692437748985182540823640700367649651112193603276388941001238545021744914878127349227324147428350849, 127352814620270581741064073869142311597126858084581060516216652266082950635262705072750085976979904016676262676936027210084518914577063899183698436464562844955931714038584228441636133475304693971219242302511775637105772928385227804835344021589932910113908144562455932991932489943966526352682811289052628434305)
sigma_ta = (1055877845737190630748039962024126733294303609922, 862944455501670227724653170357368991249521353455)
m_dec, verify = hybrid_enc_sign_verify(C_ta, sigma_ta, sk, l4, g4, p4, pk_kaiming)
print("m_dec:", int_to_str(m_dec))
print("verify:", verify)

m = 454430942186383761793031083186746476311643303704501790401838
start verification
m' = 69e289fa31a9c28f2e6645fc24d40ef7eb7c67
m_dec: He is in standard course.
verify: OK
